In [1]:
import uproot
import glob
import os
import pandas as pd
import awkward as ak


In [2]:
path_OnlyGen = "/home/ghcp/Documentos/CINVESTAV/ANALISYS_B0tomumuKstar/eficiencias/Analisys2025BdtomumuKstar_bacht2_OnlyGen_myv5/BdtoKstar2Mu_KstartoKPi_TuneCP5_13p6TeV_pythia8-evtgen/BdtomumuKstar_bacht2_OnlyGen_myv5/251005_190941/0000/*.root"
path_RecoGen    = "/home/ghcp/Documentos/CINVESTAV/ANALISYS_B0tomumuKstar/eficiencias/Analisys2025BdtomumuKstar_bacht2_RecoGen_myv1/BdtoKstar2Mu_KstartoKPi_MuFilter_TuneCP5_13p6TeV_pythia8-evtgen/BdtomumuKstar_bacht2_RecoGen_myv1/251003_201821/0000/*.root"
tree_name = "rootuple/ntuple" 


# Gen

In [13]:
# 1. Inspeccionar el primer archivo para ver las ramas
first_file = glob.glob(path_OnlyGen)[0]
with uproot.open(first_file) as f:
    tree = f[tree_name]
    print(f"--- Estructura del Árbol: {tree_name} ---")
    print(f"Entradas en este archivo: {tree.num_entries}")
    print("\n--- Ramas disponibles ---")
    tree.show() # Esto te dará el nombre y tipo de cada rama (float, int, etc.)



--- Estructura del Árbol: rootuple/ntuple ---
Entradas en este archivo: 84398

--- Ramas disponibles ---
name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
run                  | int32_t                  | AsDtype('>i4')
event                | int64_t                  | AsDtype('>i8')
lumiblock            | int32_t                  | AsDtype('>i4')
gen_b_p4             | TLorentzVector           | AsStridedObjects(Model_TLor...
gen_kstar_p4         | TLorentzVector           | AsStridedObjects(Model_TLor...
gen_kaon_p4          | TLorentzVector           | AsStridedObjects(Model_TLor...
gen_pion_p4          | TLorentzVector           | AsStridedObjects(Model_TLor...
gen_jpsi_p4          | TLorentzVector           | AsStridedObjects(Model_TLor...
gen_muon1_p4         | TLorentzVector           | AsStridedObjects(Model_TLor...
gen_muon2_p4         | TLorentzVector           | As

In [16]:
!pip install vector

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.6/179.6 kB 461.1 kB/s eta 0:00:00a 0:00:01


In [17]:
import uproot
import awkward as ak
import vector
import glob

# Registramos el comportamiento vectorial en awkward
vector.register_awkward()

path_OnlyGen = "/home/ghcp/Documentos/CINVESTAV/ANALISYS_B0tomumuKstar/eficiencias/Analisys2025BdtomumuKstar_bacht2_OnlyGen_myv5/BdtoKstar2Mu_KstartoKPi_TuneCP5_13p6TeV_pythia8-evtgen/BdtomumuKstar_bacht2_OnlyGen_myv5/251005_190941/0000/*.root"
tree_name = "rootuple/ntuple"

# Ramas crudas que necesitamos
branches = ["gen_b_p4", "gen_muon1_p4"] # Agrega las demás que necesites

try:
    first_file = glob.glob(path_OnlyGen)[0]
    with uproot.open(first_file) as f:
        # Leemos los datos crudos
        raw_data = f[tree_name].arrays(branches, library="ak", entry_stop=100) # Leemos solo 100 eventos para probar

    print("--- Transformando datos crudos a Cuadrivectores ---")

    # Función auxiliar para convertir la estructura de ROOT (fP.fX) a Vector (px, py, pz)
    def get_p4(branch_name):
        return vector.zip({
            "px": raw_data[branch_name]["fP"]["fX"],
            "py": raw_data[branch_name]["fP"]["fY"],
            "pz": raw_data[branch_name]["fP"]["fZ"],
            "E":  raw_data[branch_name]["fE"]
        })

    # Ahora creamos la variable "inteligente" para el Mesón B
    gen_b = get_p4("gen_b_p4")

    # ¡Ahora sí! Podemos ver las propiedades que querías
    print(f"\nObjeto reconstruido: {type(gen_b)}")
    print("Variables disponibles ahora (similares a la imagen):")
    
    # Imprimimos valores reales para demostrar que existen
    print(f"Pt:   {gen_b.pt[0]:.4f} (Calculado automáticamente)")
    print(f"Eta:  {gen_b.eta[0]:.4f}")
    print(f"Phi:  {gen_b.phi[0]:.4f}")
    print(f"Mass: {gen_b.mass[0]:.4f}")
    
    # Si quieres ver la lista completa de lo que puedes pedirle (como en la imagen):
    print("\nLista de atributos físicos disponibles en Python (vector):")
    print([d for d in dir(gen_b) if not d.startswith("_") and d in ['pt', 'eta', 'phi', 'mass', 'energy', 'px', 'py', 'pz']])

except Exception as e:
    print(e)

--- Transformando datos crudos a Cuadrivectores ---

Objeto reconstruido: <class 'vector.backends.awkward.MomentumArray4D'>
Variables disponibles ahora (similares a la imagen):
Pt:   5.7557 (Calculado automáticamente)
Eta:  -3.7253
Phi:  0.6919
Mass: 5.2796

Lista de atributos físicos disponibles en Python (vector):
['energy', 'eta', 'mass', 'phi', 'pt', 'px', 'py', 'pz']


# Reco

In [4]:

tree_name_guess = "ntuple" 

def explorar_muestra(path_pattern, label="Muestra"):
    print(f"\n{'='*60}")
    print(f" Explorando: {label}")
    print(f"{'='*60}")
    
    # 1. Obtener lista de archivos
    files = glob.glob(path_pattern)
    print(f" -> Directorio buscado: {os.path.dirname(path_pattern)}")
    print(f" -> Archivos encontrados: {len(files)}")
    
    if len(files) == 0:
        print(" [ERROR] No se encontraron archivos. Verifica la ruta.")
        return None

    # 2. Abrir solo el primero para ver las variables (estructura)
    first_file = files[0]
    print(f" -> Inspeccionando estructura del primer archivo:\n    {os.path.basename(first_file)}")
    
    try:
        f = uproot.open(first_file)
        # Intentar buscar el tree
        tree_keys = f.keys()
        
        # Lógica para encontrar el nombre del tree si hay varios
        target_tree = None
        for key in tree_keys:
            if tree_name_guess in key: # busca 'ntuple' o lo que definas
                target_tree = key
                break
        
        if not target_tree and len(tree_keys) > 0:
            target_tree = tree_keys[0] # Tomar el primero si no encuentra coincidencia
            
        print(f" -> Tree encontrado: '{target_tree}'")
        
        tree = f[target_tree]
        branches = tree.keys()
        
        print(f" -> Número total de ramas (variables): {len(branches)}")
        
        # 3. BÚSQUEDA DE VARIABLES CRÍTICAS (Para los ángulos)
        print("\n [DIAGNÓSTICO] ¿Tenemos las partículas hijas para los ángulos?")
        keywords = ["kaon", "pion", "K", "Pi", "trk", "dau"]
        found_vars = []
        
        for b in branches:
            # Filtramos solo las ramas que contengan palabras clave
            if any(k.lower() in b.lower() for k in keywords):
                found_vars.append(b)
                
        if len(found_vars) > 0:
            print(" -> ¡ENCONTRADAS! Ramas candidatas para hadrones:")
            for v in sorted(found_vars):
                print(f"    - {v}")
        else:
            print(" [ALERTA] No veo ramas obvias de Kaones o Piones. Revisa la lista completa.")

        # 4. Imprimir lista completa si se desea (opcional)
        # print("\n Lista completa de variables:")
        # print(branches)
        
        return branches

    except Exception as e:
        print(f" [ERROR] Al leer el archivo: {e}")
        return None

# ==========================================
# EJECUCIÓN
# ==========================================

# 1. Explorar OnlyGen (La más crítica para los ángulos)

# 2. Explorar Reco (Para confirmar que tiene lo mismo o más)
vars_reco = explorar_muestra(path_RecoGen, label="RECO (Reconstrucción)")




 Explorando: RECO (Reconstrucción)
 -> Directorio buscado: /home/ghcp/Documentos/CINVESTAV/ANALISYS_B0tomumuKstar/eficiencias/Analisys2025BdtomumuKstar_bacht2_RecoGen_myv1/BdtoKstar2Mu_KstartoKPi_MuFilter_TuneCP5_13p6TeV_pythia8-evtgen/BdtomumuKstar_bacht2_RecoGen_myv1/251003_201821/0000
 -> Archivos encontrados: 449
 -> Inspeccionando estructura del primer archivo:
    Rootuple_BdtomumuKstar_MC2022_bacht1_413.root
 -> Tree encontrado: 'rootuple/ntuple;2'
 -> Número total de ramas (variables): 91

 [DIAGNÓSTICO] ¿Tenemos las partículas hijas para los ángulos?
 -> ¡ENCONTRADAS! Ramas candidatas para hadrones:
    - AQpvTrk1
    - AQpvTrk2
    - B_TrkTrk_mass1
    - B_TrkTrk_mass2
    - B_Trk_charge1
    - B_Trk_charge2
    - B_Trk_px1
    - B_Trk_px2
    - B_Trk_py1
    - B_Trk_py2
    - B_Trk_pz1
    - B_Trk_pz2
    - FrompvTrk1
    - FrompvTrk2
    - Trk1dxy
    - Trk1dxy_e
    - Trk1dz
    - Trk1dz_e
    - Trk2dxy
    - Trk2dxy_e
    - Trk2dz
    - Trk2dz_e
    - gen_kaon_p4
    - g

# Gen Level con Filtros